<a href="https://colab.research.google.com/github/yeseul106/UROP_Software-Fault-Prediction/blob/main/UROP_JM1_64_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
import numpy as np
import pandas as pd

In [170]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [171]:
# csv 파일 읽어오기
JM1_Weka = pd.read_csv('/content/drive/MyDrive/UROP/UROP dataset/JM1_Cfs_WEKA(PROMISE).csv', encoding='cp949')
print(JM1_Weka.shape)
JM1_Weka.head(5)

(10885, 9)


,loc,v(g),ev(g),iv(g),i,lOComment,lOBlank,locCodeAndComment,defects
0,1.1,1.4,1.4,1.4,1.30,2,2,2,False
1,1.0,1.0,1.0,1.0,1.00,1,1,1,True
2,72.0,7.0,1.0,6.0,55.85,10,8,1,True
3,190.0,3.0,1.0,3.0,254.87,29,28,2,True
4,37.0,4.0,1.0,4.0,34.86,1,6,0,True


In [172]:
JM1_Weka.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'i', 'lOComment', 'lOBlank',
       'locCodeAndComment', 'defects'],
      dtype='object')

In [173]:
# 원본 csv 파일 읽어오기
JM1_original = pd.read_csv('/content/drive/MyDrive/UROP/UROP dataset/JM1_proff(PROMISE).csv', encoding='cp949')
print(JM1_original.shape)
JM1_original.head(5)

(10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,0.38,1279.39,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,1.45,4122.37,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,0.20,572.07,28,1,6,0,11,16,76,50,7,True


In [174]:
print(JM1_original.loc[JM1_original['uniq_Op'] == '?'])

       loc  v(g)  ev(g)  iv(g)  ...  total_Op  total_Opnd  branchCount  defects
143   49.0   4.0    3.0    3.0  ...         ?           ?            ?     True
358   27.0   5.0    3.0    3.0  ...         ?           ?            ?     True
1598  76.0  18.0   11.0   11.0  ...         ?           ?            ?     True
4214   6.0   1.0    1.0    1.0  ...         ?           ?            ?    False
8279   9.0  13.0   10.0    4.0  ...         ?           ?            ?    False

[5 rows x 22 columns]


In [175]:
# 이상 값이 있는 행 삭제
JM1_original = JM1_original.drop([143,358,1598,4214,8279], axis=0)

In [176]:
print(JM1_original.shape)

(10880, 22)


# **Feature Selection을 한 경우**

- hidden layer : 3개
- hidden layer 노드 수 : 64개
- Dropout layer : 3개, 비율은 0.5
- 각 층의 활성화 함수: relu
- 출력층 활성화 함수: sigmoid
- optimizer 함수: adam
- batch size : 10

In [177]:
# 입력 데이터
input_data = JM1_original.iloc[:,:21]
print("input_data shape : ",input_data.shape)

input_data shape :  (10880, 21)


In [178]:
input_data = np.asarray(input_data,dtype=np.float)

In [179]:
input_data[0]

array([1.1, 1.4, 1.4, 1.4, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 2. ,
       2. , 2. , 2. , 1.2, 1.2, 1.2, 1.2, 1.4])

In [180]:
input_data.shape

(10880, 21)

In [181]:
# 정답 레이블
y_label = JM1_original.iloc[:,21].to_numpy()
print("y_label shape : ",y_label.shape)

y_label shape :  (10880,)


In [182]:
y_label[0]

False

In [183]:
# 정답 레이블 바꾸기
for i in range(len(y_label)):
  if y_label[i]:
    y_label[i] = False
  else:
    y_label[i] = True

In [184]:
# 레이블 벡터화 
y_label = np.asarray(y_label).astype('float32')

In [185]:
y_label.shape

(10880,)

In [186]:
from sklearn import preprocessing

# 정규화 0-1로 하기
data_normalizer = preprocessing.MinMaxScaler() # 데이터를 0~1 범위로 점철되게 하는 함수 call
data_normalized = data_normalizer.fit_transform(input_data) # 데이터를 0~1 범위로 점철되게 함수 수행
data_normalized.shape

(10880, 21)

In [187]:
data_normalized[0]

array([2.90613194e-05, 8.52878465e-04, 2.43902439e-03, 9.97506234e-04,
       1.54010188e-04, 1.60805353e-05, 1.00000000e+00, 3.10856050e-03,
       2.28158237e-03, 4.18278349e-08, 4.82374768e-02, 7.52901028e-07,
       7.08215297e-04, 5.81395349e-03, 4.47427293e-03, 1.85185185e-02,
       2.91970803e-03, 1.16959064e-03, 2.21402214e-04, 3.97219464e-04,
       4.84848485e-04])

# **10-fold cross-valid**

In [188]:
from sklearn.model_selection import StratifiedKFold
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

# 10개의 파일로 쪼갬
n_fold = 10
seed = 0
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

#빈 accuracy 배열
accuracy = []
loss = []

In [189]:
from typing import Pattern
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

In [190]:
for train, test in skf.split(data_normalized, y_label):
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(21,)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
  hist = model.fit(data_normalized[train], y_label[train], epochs=600, batch_size=10, verbose=1, callbacks=[early_stopping_callback])
  k_accuracy = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[1])
  k_loss = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[0])
  accuracy.append(k_accuracy)
  loss.append(k_loss)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
980/980 [==============================] - 3s 3ms/step - loss: 0.4423 - accuracy: 0.8124
Epoch 368/600
980/980 [==============================] - 3s 3ms/step - loss: 0.4385 - accuracy: 0.8124
Epoch 369/600
980/980 [==============================] - 4s 4ms/step - loss: 0.4400 - accuracy: 0.8140
Epoch 370/600
980/980 [==============================] - 4s 4ms/step - loss: 0.4400 - accuracy: 0.8105
Epoch 371/600
980/980 [==============================] - 3s 3ms/step - loss: 0.4378 - accuracy: 0.8144
Epoch 372/600
980/980 [==============================] - 3s 3ms/step - loss: 0.4417 - accuracy: 0.8133
Epoch 373/600
980/980 [==============================] - 3s 3ms/step - loss: 0.4429 - accuracy: 0.8122
Epoch 374/600
980/980 [==============================] - 3s 4ms/step - loss: 0.4412 - accuracy: 0.8110
Epoch 375/600
980/980 [==============================] - 4s 4ms/step - loss: 0.4426 - accuracy: 0.8112
Epoch 376/600
980/980 [============================

In [191]:
print("\n %.f fold accuracy:" % n_fold, accuracy)
print("모든 fold의 평균 auc: ", (float(accuracy[0]) + float(accuracy[1]) + float(accuracy[2]) + float(accuracy[3]) + float(accuracy[4])+ float(accuracy[5])+ float(accuracy[6])+ float(accuracy[7])+ float(accuracy[8])+ float(accuracy[9])) / 10)

print("\n %.f fold loss:" % n_fold, loss)
print("모든 fold의 평균 loss: ", (float(loss[0]) + float(loss[1]) + float(loss[2]) + float(loss[3]) + float(loss[4]) + float(loss[5]) + float(loss[6]) + float(loss[7]) + float(loss[8]) + float(loss[9])) / 10)


 10 fold accuracy: ['0.8079', '0.8134', '0.8318', '0.8024', '0.8153', '0.8180', '0.8107', '0.8143', '0.8153', '0.8171']
모든 fold의 평균 auc:  0.81462

 10 fold loss: ['0.4374', '0.5758', '0.4167', '0.4537', '0.4167', '0.4414', '0.4509', '0.6342', '0.4782', '0.4565']
모든 fold의 평균 loss:  0.47614999999999996
